última actualización: 11.05.2024

//////////////////////////////////////////////////////////////////////

categoría = "Algebra conmutativa"

info="AUTOR: Ainhoa Hernández Delgado [ainhoahdezdelgado@gmail.com](mailto:ainhoahdezdelgado@gmail.com)

DESCRIPCION: Un algoritmo que calcula el rango de Waring para cualquier forma homogénea y una descomposición de Waring para los casos más sencillos haciendo uso de polinomios apolares y del ideal anulador. El proceso se basa en el trabajo de fin de grado: "El problema de Waring para enteros y polinomios" por Ainhoa Hernández Delgado.

PROCEDIMIENTOS:

Calc\_coef\(pol\): Obtiene los coeficientes de un polinomio en un anillo con coeficientes simbólicos y los convierte en una lista útil en otros anillos de polinomios.

Descomp\(pol,d,coef\_og\); Calcula la descomposición de un polinomio dado su grado, coeficientes y un polinomio apolar al mismo.

Limpiar\(K,raiz\); Toma una lista y una matriz de complejos y elimina los errores de aproximación cometidos en sus valores, considerando como tal todo número \(o su parte real o compleja\) con valor absoluto inferior a 1e\-10.

Waring\(f\): Toma un polinomio homogéneo de grado d y devuelve su rango de Waring y, si es posible, una descomposición minimal como suma de potencias d\-ésimas de formas lineales."



In [4]:
def Calc_coef(pol,d):
    '''USO:   DCalc_coef(pol); pol es un polinomio en dos variables con coeficientes complejos.
    DEVUELVE: Una lista con los coeficientes de pol, de tal forma que el coeficiente de x^iy^d-i se corresponde con coef_og[i].
    ASUME:  pol es un polinomio homogéneo.
    '''
    R.<x,y> = PolynomialRing(SR,2)
    coef_og=[]
    for i in range(0,d+1):
        coef_og.append(pol.coefficient({x: i,y:d-i}))
    return coef_og

In [5]:
def Descomp(pol,d,coef_og):
    '''
    USO:   Descomp(pol,d,coef_og); pol es un polinomio en dos variables con coeficientes complejos, d es un entero positivo y coef_og es una       lista de números complejos.
    DEVUELVE: Un string con una descomposición minimal de Waring si ha sido capaz de calcularla y un string con la frase [NO SE ENCUENTRA           DISPONIBLE] en caso contrario.
    ASUME:  pol es un polinomio apolar al polinomio homogéneo de grado d y con coeficientes en la lista coef_og, de tal forma que el               coeficiente de x^iy^d-i se corresponde con coef_og[i].
    '''
    W=pol.degree()
    x=var('x')
    raiz=solve(SR(pol.subs(y=1)),x) #Obtenemos las raíces de phi
    if any(raiz[i].lhs()==0 for i in range(len(raiz))):
        desc='[NO SE ENCUENTRA DISPONIBLE]'
    else:
        C = Matrix(SR,d+1,d+1) #Creamos la matriz para hallar las constantes completando con la identidad
        D = Matrix(SR,d+1,1) #Creamos el vector para resolver el sistema Cx=D de constantes
        if pol.coefficient({x: W,y:0})!=0: #Separamos los casos en los que la forma de Waring tiene x^d
            for j in range(0,W):
                for i in range(0,d+1):
                    C[i,j]=(raiz[j].rhs()^i).n()
            for j in range(W,d+1):
                if any(abs(r.rhs().n())<0.0001 for r in raiz): #Para completar con la Id vemos cuando el 0 es raíz y aparece el vector canónico
                    for i in range(0,d+1):
                        if j-i==W-1:
                            C[i,j]=1
                else:
                    for i in range(0,d+1):
                        if j-i==W:
                            C[i,j]=1
            for i in range(0,d+1):
                D[i,0]=coef_og[i]/binomial(d,i) #Nótese que coef_og[i] se corresponde con el coef de x^i*y^(d-i) y (d i)=(d d-i)
            K=C.inverse()*D #Las primeras constantes de este vector son las que nos interesan
            K,raiz=Limpiar(K,raiz)
            desc='('+str(K[0,0])+')*(('+str(raiz[0])+')*x+y)^'+str(d)
            for i in range(1,len(raiz)):
                desc=desc+'+('+str(K[i,0])+')*(('+str(raiz[i])+')*x+y)^'+str(d)
        else:
            for j in range(0,W-1):
                for i in range(0,d+1):
                    C[i,j]=(raiz[j].rhs()^i).n()
            for j in range(W-1,d+1):
                for i in range(0,d+1):
                    if j-i==W-1:
                        C[d-i,j]=1
            for i in range(0,d+1):
                D[i,0]=coef_og[i]/binomial(d,i) #Nótese que coef_og[i] se corresponde con el coef de x^i*y^(d-i) y (d i)=(d d-i)
            K=C.inverse()*D #Las primeras constantes de este vector son las que nos interesan
            K,raiz=Limpiar(K,raiz)
            desc='('+str(K[len(raiz),0].n())+')*(x+0*y)^'+str(d)
            for i in range(0,len(raiz)):
                desc=desc+'+('+str(K[i,0])+')*(('+str(raiz[i])+')*x+y)^'+str(d)
    return desc

In [6]:
def Limpiar(K,raiz):
    '''
    USO:  Limpiar(K,raiz); K es una matriz de complejos y raiz una lista de complejos.
    DEVUELVE: Una matriz y una lista de complejos en las que se han eliminado los números (o sus partes real o imaginaria) con valor absoluto       inferior a 1e-10.
    ASUME: K tiene al menos len(raiz) filas.
    '''
    K2=Matrix(SR,len(raiz)+1,1)
    raiz2=vector(SR,len(raiz))
    for i in range(0,len(raiz)):
        if K[i,0].real().abs()<1e-10:
            K2[i,0]=K[i,0].imag().n()*I
        elif K[i,0].imag().abs()<1e-10:
            K2[i,0]=K[i,0].real().n()
        else:
            K2[i,0]=K[i,0].n()
        if raiz[i].rhs().n().abs()<1e-10:
            raiz2[i]=0
        elif raiz[i].rhs().n().real().abs()<1e-10:
            raiz2[i]=raiz[i].rhs().n().imag()*I
        elif raiz[i].rhs().n().imag().abs()<1e-10:
            raiz2[i]=raiz[i].rhs().n().real()
        else:
            raiz2[i]=raiz[i].rhs().n()
    j=len(raiz)
    if K[j,0].real().abs()<1e-10:
        K2[j,0]=K[j,0].imag().n()*I
    elif K[j,0].imag().abs()<1e-10:
        K2[j,0]=K[j,0].real().n()
    else:
        K2[j,0]=K[j,0].n()
    return K2,raiz2

In [11]:
def Waring(f):
    '''
    USO:  Waring(f); f es un polinomio en dos variables con coeficientes complejos.
    DEVUELVE: Un mensaje por pantalla en el que se declara el rango de Waring, una descomposición minimal para f si fue posible calcularla e       información sobre las raíces del polinomio apolar que se ha utilizado para el cálculo de la descomposición.
    ASUME: f es un polinomio homogéneo.
    '''
    R.<x, y> = PolynomialRing(CC, 2)
    'Sacamos los coeficientes de f en función de la base B_d,i'
    d=f.degree()
    coef_og=Calc_coef(f,d)
    coef=[]
    for i in range(0,d+1):
        coef.append(coef_og[i]*factorial(i)*factorial(d-i))
    'Hallamos la matriz A_u de menor dimensión cuyo rango no sea máximo'
    u=0
    rango=1
    while rango==u+1:
        u=u+1
        A = Matrix(SR,d-u+1,u+1)
        for i in range(0,d-u+1):
            for j in range(0,u+1):
                A[i,j]=coef[i+j]
        rango=A.rank()
    'Resolvemos el sistema AX=0 para obtener los coeficentes de phi y construimos phi'
    X=vector([var("a_{}".format(i)) for i in (0..u)])
    B=A*X
    system = [eq == 0 for eq in B]
    sol=solve(system,[var("a_{}".format(i)) for i in (0..u)])
    if A.rank()==u:
        for i in range(0,len(sol[0])): #Evaluamos los parametros libres
            sol[0][i]=sol[0][i].subs({j: 1 for j in sol[0][i].rhs().variables()})
    else:
        v=[]
        i=-1
        while v==[]:
            i=i+1
            v.append(sol[0][i].rhs().variables()[0])
        for i in range(0,len(sol[0])): #Evaluamos los parametros libres
            while sol[0][i].rhs().variables()!=():
                if sol[0][i].rhs().variables()[0]==v[0]:
                    sol[0][i]=sol[0][i].subs({sol[0][i].rhs().variables()[0]:1})
                else:
                    sol[0][i]=sol[0][i].subs({sol[0][i].rhs().variables()[0]:0})
    phi=0
    for i in range(0,len(sol[0])):
        phi=phi+QQbar(sol[0][i].rhs())*x^i*y^(u-i)
    'Comprobamos si phi es libre de cuadrados'
    if gcd(phi.subs(y=1),phi.subs(y=1).derivative(x))==1 and (sol[0][u].rhs()!=0 or sol[0][u-1].rhs()!=0):
        W=u
        desc=Descomp(phi,d,coef_og)
        print('Phi no tiene factores múltiples, el rango de Waring es '+str(W)+' y la descomposición es '+desc)
    else:
        W=d+2-u
        if A.rank()==u:
            A2 = Matrix(SR,d-W+1,W+1)
            for i in range(0,d-W+1):
                for j in range(0,W+1):
                    A2[i,j]=coef[i+j]
            X2=vector([var("a_{}".format(i)) for i in (0..W)])
            B2=A2*X2
            system2 = [eq == 0 for eq in B2]
            sol2=solve(system2,[var("a_{}".format(i)) for i in (0..W)])
            for i in range(0,len(sol2[0])): #Evaluamos los parametros libres
                sol2[0][i]=sol2[0][i].subs({j: 1 for j in sol2[0][i].rhs().variables()})
            psi=0
            for i in range(0,len(sol2[0])):
                psi=psi+QQbar(sol2[0][i].rhs())*x^i*y^(W-i)
        else:
            sol2=solve(system,[var("a_{}".format(i)) for i in (0..u)])
            v2=[]
            i=-1
            while v2==[]:
                i=i+1
                v2.append(sol2[0][i].rhs().variables()[0])
            for i in range(0,len(sol2[0])): #Evaluamos los parametros libres
                while sol2[0][i].rhs().variables()!=():
                    if sol2[0][i].rhs().variables()[0]==v2[0]:
                        sol2[0][i]=sol2[0][i].subs({sol2[0][i].rhs().variables()[0]:0})
                    else:
                        sol2[0][i]=sol2[0][i].subs({sol2[0][i].rhs().variables()[0]:1})
            psi=0
            for i in range(0,len(sol2[0])):
                psi=psi+QQbar(sol2[0][i].rhs())*x^i*y^(W-i)
        if gcd(psi.subs(y=1),psi.subs(y=1).derivative(x))==1 and (sol2[0][W].rhs()!=0 or sol2[0][W-1].rhs()!=0):
            desc=Descomp(psi,d,coef_og)
            print('Phi tiene factores múltiples, el rango de Waring es '+str(W)+' y la descomposición es '+desc)
        else:
            a=0
            pol=phi+a*y^(psi.degree()-phi.degree())*psi
            while gcd(pol.subs(y=1),pol.subs(y=1).derivative(x))!=1 or (pol.coefficient({x:W,y:0})==0 and pol.coefficient({x:W-1,y:1})==0):
                a=a+1
                pol=phi+a*y^(psi.degree()-phi.degree())*psi
            desc=Descomp(pol,d,coef_og)
            print('Phi y psi tienen factores múltiples, utilizamos el polinomio dado por phi+'+str(a)+'y^'+str(psi.degree()-phi.degree())+'*psi, el rango de Waring es '+str(W)+' y la descomposición es '+desc)

### Ejemplos de uso.



In [12]:
R.<x,y> = PolynomialRing(SR, 2)
f = x^2 + x*y
Waring(f)

Phi tiene factores múltiples, el rango de Waring es 2 y la descomposición es (-0.250000000000000)*((0)*x+y)^2+(0.250000000000000)*((2.00000000000000)*x+y)^2


In [13]:
R.<x,y> = PolynomialRing(SR, 2)
f = x*y + y^2
Waring(f)

Phi no tiene factores múltiples, el rango de Waring es 2 y la descomposición es (-0.250000000000000)*(x+0*y)^2+(1.00000000000000)*((0.500000000000000)*x+y)^2


In [14]:
R.<x,y> = PolynomialRing(SR, 2)
f = x*y
Waring(f)

Phi y psi tienen factores múltiples, utilizamos el polinomio dado por phi+1y^0*psi, el rango de Waring es 2 y la descomposición es (0.250000000000000*I)*((-1.00000000000000*I)*x+y)^2+(-0.250000000000000*I)*((1.00000000000000*I)*x+y)^2


In [15]:
R.<x, y> = PolynomialRing(SR, 2)
f = 9*x^3 + (-15)*x^2*y + 9*x*y^2 + (-2)*y^3
Waring(f)

Phi no tiene factores múltiples, el rango de Waring es 2 y la descomposición es (-1.00000000000000)*((-2.00000000000000)*x+y)^3+(-1.00000000000000)*((-1.00000000000000)*x+y)^3


In [16]:
R.<x, y> = PolynomialRing(SR, 2)
f = 2*x^5 + 5*x^4*y + 10*x^3*y^2 + 10*x^2*y^3 + 5*x*y^4 + 2*y^5
Waring(f)

Phi no tiene factores múltiples, el rango de Waring es 3 y la descomposición es (1.00000000000000)*(x+0*y)^5+(1.00000000000000)*((0)*x+y)^5+(1.00000000000000)*((1.00000000000000)*x+y)^5


In [17]:
R.<x,y> = PolynomialRing(SR, 2)
f = 3*x^6 + 18*x^5*y + 165*x^4*y^2 + 540*x^3*y^3 + 1245*x^2*y^4 + 1458*x*y^5 + 731*y^6
Waring(f)

Phi no tiene factores múltiples, el rango de Waring es 3 y la descomposición es (729.000000000000)*((0.333333333333333)*x+y)^6+(0.999999999999996)*((-1.00000000000000)*x+y)^6+(0.999999999999996)*((1.00000000000000)*x+y)^6


In [18]:
R.<x, y> = PolynomialRing(SR, 2)
f = x^5*y
Waring(f)

Phi tiene factores múltiples, el rango de Waring es 6 y la descomposición es (0.0321555407294946 + 0.0279468317173761*I)*((-0.727136084491197 - 0.430014288329716*I)*x+y)^6+(0.0321555407294946 - 0.0279468317173761*I)*((-0.727136084491197 + 0.430014288329716*I)*x+y)^6+(-0.0154888740628279 - 0.0114622687183189*I)*((0.727136084491197 - 0.934099289460529*I)*x+y)^6+(-0.0154888740628279 + 0.0114622687183189*I)*((0.727136084491197 + 0.934099289460529*I)*x+y)^6+(-0.0166666666666667 + 0.0333333333333334*I)*((-1.00000000000000*I)*x+y)^6+(-0.0166666666666667 - 0.0333333333333333*I)*((1.00000000000000*I)*x+y)^6
